In [1]:
!pip install -qU bitsandbytes peft trl accelerate datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.5/532.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 7.8 MB/s eta 0:00:00


In [2]:
import os
import transformers
import torch
from datasets import load_dataset
from google.colab import userdata
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [62]:
model_id = "google/gemma-2b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )

In [63]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ['HF_TOKEN'])

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                             device_map = {"":0},
                                             token = os.environ['HF_TOKEN'])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [64]:
text = "Explain the Transformer Architecture?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Explain the Transformer Architecture?

The Transformer architecture is a type of neural network that is used for sequence-to-sequence tasks. It is a type of encoder-decoder architecture, which means that it has an encoder and a decoder. The encoder takes in a sequence of tokens and encodes them into a fixed-length representation, while the decoder takes in this representation and generates a sequence of tokens.

The Transformer architecture is based on the idea of self-attention, which is a type of attention mechanism that allows the network to


In [65]:
os.environ['WANDB_DISABLED'] = "False"

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)

In [66]:
data = load_dataset("Abirate/english_quotes")

In [67]:
data['train'][100]

{'quote': '“I\'m in love with you," he said quietly."Augustus," I said."I am," he said. He was staring at me, and I could see the corners of his eyes crinkling. "I\'m in love with you, and I\'m not in the business of denying myself the simple pleasure of saying true things. I\'m in love with you, and I know that love is just a shout into the void, and that oblivion is inevitable, and that we\'re all doomed and that there will come a day when all our labor has been returned to dust, and I know the sun will swallow the only earth we\'ll ever have, and I am in love with you.”',
 'author': 'John Green,',
 'tags': ['doomed', 'inevitable', 'love', 'oblivion', 'pleasure', 'simple']}

In [68]:
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)

In [69]:
def formatting_func(example):
  text = f"Quote: {example['quote']} \nAuthor: {example['author']}"
  return [text]

print(formatting_func(data['train'][10]))

["Quote: “In three words I can sum up everything I've learned about life: it goes on.” \nAuthor: Robert Frost"]


In [70]:
trainer = SFTTrainer(
    model = model,
    train_dataset = data['train'],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        bf16 = True,
        logging_steps = 1,
        output_dir = "outputs",
        optim = "paged_adamw_8bit"
    ),
    peft_config = lora_config,
    formatting_func = formatting_func
)

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:2111: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(


In [71]:
trainer.train()

Step,Training Loss
1,2.561400
2,1.628800
3,2.491200
4,2.766600
5,2.302300
6,2.488900
7,2.898900
8,2.270700
9,3.209500
10,2.240800


wandb: WARNING URL not available in offline run


TrainOutput(global_step=100, training_loss=2.0633102411031725, metrics={'train_runtime': 302.6759, 'train_samples_per_second': 1.322, 'train_steps_per_second': 0.33, 'total_flos': 189744345784320.0, 'train_loss': 2.0633102411031725})

In [72]:
text = "Quote: A women is like a tea bag"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: A women is like a tea bag. You never know how strong she is until she gets in hot water.

I am a woman


In [73]:
text = "Quote: The opposite of love is not hate"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: The opposite of love is not hate, it's indifference.

The opposite of love is indifference.

The opposite of love is
